In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
def excess_return(stock, indice, train=True):
    '''
    input = archivo con informacion de precios con fecha en txt
    output = dataframe con la fecha (formato datetime) y cierre
    '''
    # armo el nombre del archivo a leer de los parametros
    dir_name = '../data/original'
    
    if train:
        stock_name  = stock  + '_train.txt'
        indice_name = indice + '_train.txt'
    else:
        stock_name  = stock  + '_test.txt'
        indice_name = indice + '_test.txt'
            
    stockname  = os.path.join(dir_name, stock_name)
    indicename = os.path.join(dir_name, indice_name)
    
    # leo los archivos y le cambio los nombres a las columnas
    stk = pd.read_csv(stockname, usecols=['Date', 'Adj Close'])
    ind = pd.read_csv(indicename,usecols=['Date', 'Adj Close'])
    stk.columns = ['date', stock]
    ind.columns = ['date', indice]
    
    # mergeo los df con los precios de la acción y el índice
    data = pd.merge(stk, ind, on='date')
    
    # paso la fecha a formato datetime
    data['date'] = pd.to_datetime(data.date)
    
    # armo un df con el total de los días en el rango de fechas de los archivos provistos
    dias_totales = pd.DataFrame(pd.date_range(data.date.min(), data.date.max(), freq='D'), columns=['date'])
    
    # mergeo para buscar los días sin valor en precios (fin de semana y feriados)
    data = pd.merge(dias_totales, data, on='date', how='left')
    
    # interpolo los nans con función cuadrática
    data = data.interpolate(method='quadratic')
    
    # calculo el retorno logarítmico
    data['log_ret NFLX'] = np.log(data.NFLX / data.NFLX.shift(1))
    data['log_ret NDX'] = np.log(data.NDX / data.NDX.shift(1))
    data['exc_ret'] = (data['log_ret NFLX'] - data['log_ret NDX']) > 0
    
    # armo el df para retornar con la fecha y el excess return de la acción eliminando
    # la primera observación (sin retorno calculado)
    data = data.loc[1: , ['date', 'exc_ret']]

    return data

In [3]:
prices_train = excess_return(stock='NFLX', indice='NDX', train=True)
prices_test  = excess_return(stock='NFLX', indice='NDX', train=False)

In [4]:
prices_train.to_csv('../data/prices_train.csv')
prices_test.to_csv('../data/prices_test.csv')